<link rel='stylesheet' href='../assets/css/main.css'/>

[<< back to main index](../README.md) 

# Lab 4.1 : Spark UDF


### Overview
Using User Defined Functions with Spark Dataframes

### Depends On 
None

### Run time
15-20 mins


In [1]:
print('Spark UI running on http://YOURIPADDRESS:' + sc.uiWebUrl.split(':')[2])

Spark UI running on http://YOURIPADDRESS:4040



## STEP 1: Load MT Cars data

In [3]:
mtcarsDF = spark.read.csv("/data/cars/mtcars_header.csv", header="true")
print(mtcarsDF)

DataFrame[model: string, mpg: string, cyl: string, disp: string, hp: string, drat: string, wt: string, qsec: string, vs: string, am: string, gear: string, carb: string]




## STEP 2 : Inspecting The Dataframe

**==> Print the schema of data frame**     


In [4]:
# TODO: Print the schema
# Hint : `printSchema()`

# mtcarsDF.????()
mtcarsDF.printSchema()

root
 |-- model: string (nullable = true)
 |-- mpg: string (nullable = true)
 |-- cyl: string (nullable = true)
 |-- disp: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- drat: string (nullable = true)
 |-- wt: string (nullable = true)
 |-- qsec: string (nullable = true)
 |-- vs: string (nullable = true)
 |-- am: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- carb: string (nullable = true)



Your output may look like this:

```
root
 |-- model: string (nullable = true)
 |-- mpg: string (nullable = true)
 |-- cyl: string (nullable = true)
 |-- disp: string (nullable = true)
 |-- hp: string (nullable = true)
 |-- drat: string (nullable = true)
 |-- wt: string (nullable = true)
 |-- qsec: string (nullable = true)
 |-- vs: string (nullable = true)
 |-- am: string (nullable = true)
 |-- gear: string (nullable = true)
 |-- carb: string (nullable = true)

```

**==> Print / Dump the data contained within dataframe**  

In [5]:
# TODO: Print/Dump the data
# Hint : show()
#mtcarsDF.???()
mtcarsDF.show()

+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|          Mazda RX4|  21|  6|  160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|  21|  6|  160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|  108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|  258|110|3.08|3.215|19.44|  1|  0|   3|   1|
|  Hornet Sportabout|18.7|  8|  360|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|            Valiant|18.1|  6|  225|105|2.76| 3.46|20.22|  1|  0|   3|   1|
|         Duster 360|14.3|  8|  360|245|3.21| 3.57|15.84|  0|  0|   3|   4|
|          Merc 240D|24.4|  4|146.7| 62|3.69| 3.19|   20|  1|  0|   4|   2|
|           Merc 230|22.8|  4|140.8| 95|3.92| 3.15| 22.9|  1|  0|   4|   2|
|           Merc 280|19.2|  6|167.6|123|3.92| 3.44| 18.3|  1|  0|   4|   4|
|          M

Your output may look like this:

```
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              model| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-------------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|          Mazda RX4|  21|  6|  160|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|      Mazda RX4 Wag|  21|  6|  160|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|         Datsun 710|22.8|  4|  108| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|     Hornet 4 Drive|21.4|  6|  258|110|3.08|3.215|19.44|  1|  0|   3|   1|
...

```

In [23]:
mpgDF = mtcarsDF.select("model", mtcarsDF.mpg.cast("float"))
mpgDF.show()

+-------------------+----+
|              model| mpg|
+-------------------+----+
|          Mazda RX4|21.0|
|      Mazda RX4 Wag|21.0|
|         Datsun 710|22.8|
|     Hornet 4 Drive|21.4|
|  Hornet Sportabout|18.7|
|            Valiant|18.1|
|         Duster 360|14.3|
|          Merc 240D|24.4|
|           Merc 230|22.8|
|           Merc 280|19.2|
|          Merc 280C|17.8|
|         Merc 450SE|16.4|
|         Merc 450SL|17.3|
|        Merc 450SLC|15.2|
| Cadillac Fleetwood|10.4|
|Lincoln Continental|10.4|
|  Chrysler Imperial|14.7|
|           Fiat 128|32.4|
|        Honda Civic|30.4|
|     Toyota Corolla|33.9|
+-------------------+----+
only showing top 20 rows



In [24]:
#Register dataframe as a table
mpgDF.createOrReplaceTempView("mpgDF")
print("created mpgDF temp table")

created mpgDF temp table



## STEP 3 : Define the UDF

**==> Create a UDF to convert mpg to kml i.e. kilometers/litre**

In [25]:
def mpg_to_kml(mpg):
    kml = mpg * 1.6 / 3.7854
    return kml

In [27]:
#register the UDF with Spark SQL
from pyspark.sql.types import FloatType
sqlContext.udf.register("MPG_to_KmPerLiter", mpg_to_kml, FloatType())

<function __main__.mpg_to_kml(mpg)>

In [28]:

# Use the UDF in SQL

resultDF = sqlContext.sql("select model, MPG_to_KmPerLiter(mpg) from mpgDF")
resultDF.show()

+-------------------+----------------------+
|              model|MPG_to_KmPerLiter(mpg)|
+-------------------+----------------------+
|          Mazda RX4|              8.876208|
|      Mazda RX4 Wag|              8.876208|
|         Datsun 710|              9.637026|
|     Hornet 4 Drive|             9.0452795|
|  Hornet Sportabout|             7.9040527|
|            Valiant|             7.6504464|
|         Duster 360|             6.0442753|
|          Merc 240D|             10.313309|
|           Merc 230|              9.637026|
|           Merc 280|              8.115391|
|          Merc 280C|              7.523643|
|         Merc 450SE|              6.931896|
|         Merc 450SL|              7.312305|
|        Merc 450SLC|              6.424684|
| Cadillac Fleetwood|             4.3958364|
|Lincoln Continental|             4.3958364|
|  Chrysler Imperial|              6.213346|
|           Fiat 128|             13.694722|
|        Honda Civic|             12.849368|
|     Toyo

## Step 5 : Use the UDF in DataFrame


In [30]:
from pyspark.sql.functions import udf
convert_udf = udf(mpg_to_kml, FloatType())

resultsDF = mpgDF.select("model", convert_udf("mpg").alias("km_per_liter"))
display(resultsDF)
resultsDF.show()



DataFrame[model: string, km_per_liter: float]

+-------------------+------------+
|              model|km_per_liter|
+-------------------+------------+
|          Mazda RX4|    8.876208|
|      Mazda RX4 Wag|    8.876208|
|         Datsun 710|    9.637026|
|     Hornet 4 Drive|   9.0452795|
|  Hornet Sportabout|   7.9040527|
|            Valiant|   7.6504464|
|         Duster 360|   6.0442753|
|          Merc 240D|   10.313309|
|           Merc 230|    9.637026|
|           Merc 280|    8.115391|
|          Merc 280C|    7.523643|
|         Merc 450SE|    6.931896|
|         Merc 450SL|    7.312305|
|        Merc 450SLC|    6.424684|
| Cadillac Fleetwood|   4.3958364|
|Lincoln Continental|   4.3958364|
|  Chrysler Imperial|    6.213346|
|           Fiat 128|   13.694722|
|        Honda Civic|   12.849368|
|     Toyota Corolla|   14.328737|
+-------------------+------------+
only showing top 20 rows

